In [31]:
import findspark
findspark.init("C:\spark")

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

spark = SparkSession.builder. \
master("local[4]"). \
appName("Df-clean"). \
config("spark.driver.memory","2g"). \
config("spark.executor.memory","4g"). \
getOrCreate()

sc = spark.sparkContext

In [33]:
df =spark.read \
.option("sep",",") \
.option("header" , "True") \
.option("inferSchema" , "True") \
.csv("Data\\simple_dirty_data.csv")

In [34]:
df.show()

+------+---------+---+--------+-----------+-----------+-----------+--------------------+
|sirano|     isim|yas|cinsiyet|     meslek|      sehir|aylik_gelir|            mal_mulk|
+------+---------+---+--------+-----------+-----------+-----------+--------------------+
|     1|    Cemal| 35|       E|       Isci|     Ankara|     3500.0|               araba|
|     2|   ceyda | 42|       K|      Memur|    Kayseri|     4200.0|            araba|ev|
|     3|    Timur| 30|    null|   Müzüsyen|Istanbul   |     9000.0|     araba|ev|yazlık|
|     4|   Burcu | 29|       K|Pazarlamacı|     Ankara|     4200.0|               araba|
|     5|  Yasemin| 23|       K|Pazarlamaci|      Bursa|     4800.0|               araba|
|     6|      Ali| 33|       E|      Memur|     Ankara|     4250.0|                  ev|
|     7|    Dilek| 29|       K|Pazarlamaci|   Istanbul|     7300.0|        araba|yazlık|
|     8|    Murat| 31|       E|   Müzüsyen|   Istanbul|    12000.0|araba|ev|dükkan|y...|
|     9|    Ahmet| 33

In [35]:
from pyspark.sql.functions import *

In [36]:
df2 = df \
.withColumn("isim",trim(initcap(df.isim))) \
.withColumn("cinsiyet" , when(df.cinsiyet.isNull() ,"U").otherwise(df.cinsiyet)) \
.withColumn("sehir" , when(df.sehir.isNull() , "BİLİNMİYOR").otherwise(trim(upper(df.sehir))))

df2.show(10)

+------+--------+---+--------+-----------+--------+-----------+--------------------+
|sirano|    isim|yas|cinsiyet|     meslek|   sehir|aylik_gelir|            mal_mulk|
+------+--------+---+--------+-----------+--------+-----------+--------------------+
|     1|   Cemal| 35|       E|       Isci|  ANKARA|     3500.0|               araba|
|     2|   Ceyda| 42|       K|      Memur| KAYSERI|     4200.0|            araba|ev|
|     3|   Timur| 30|       U|   Müzüsyen|ISTANBUL|     9000.0|     araba|ev|yazlık|
|     4|   Burcu| 29|       K|Pazarlamacı|  ANKARA|     4200.0|               araba|
|     5| Yasemin| 23|       K|Pazarlamaci|   BURSA|     4800.0|               araba|
|     6|     Ali| 33|       E|      Memur|  ANKARA|     4250.0|                  ev|
|     7|   Dilek| 29|       K|Pazarlamaci|ISTANBUL|     7300.0|        araba|yazlık|
|     8|   Murat| 31|       E|   Müzüsyen|ISTANBUL|    12000.0|araba|ev|dükkan|y...|
|     9|   Ahmet| 33|       E|     Doktor|  ANKARA|   180000.0|  

## temizlik bitti lets write to disk

In [37]:
df2 \
.coalesce(1) \
.write \
.mode("overwrite") \
.option("sep" , ",") \
.option("header", "True") \
.csv("Data\\simple_dirty_data")

In [38]:
df3 = spark.read \
.option("sep",",") \
.option("header" , "True") \
.option("inferSchema" , "True") \
.csv("Data\\simple_dirty_data")


In [39]:
df3.show(5)

+------+-------+---+--------+-----------+--------+-----------+---------------+
|sirano|   isim|yas|cinsiyet|     meslek|   sehir|aylik_gelir|       mal_mulk|
+------+-------+---+--------+-----------+--------+-----------+---------------+
|     1|  Cemal| 35|       E|       Isci|  ANKARA|     3500.0|          araba|
|     2|  Ceyda| 42|       K|      Memur| KAYSERI|     4200.0|       araba|ev|
|     3|  Timur| 30|       U|   Müzüsyen|ISTANBUL|     9000.0|araba|ev|yazlık|
|     4|  Burcu| 29|       K|Pazarlamacı|  ANKARA|     4200.0|          araba|
|     5|Yasemin| 23|       K|Pazarlamaci|   BURSA|     4800.0|          araba|
+------+-------+---+--------+-----------+--------+-----------+---------------+
only showing top 5 rows

